In [2]:
# export_to_torchscript.py

import torch
from interpolator import Interpolator

# ==== 路径设置 ====
ckpt_path = "film_epoch10.pth"     # 请确保路径正确
export_path = "film_sft_last.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = Interpolator().to(device)
# ==== 构造模型结构 ====
#model = load_model_from_pth(pth_path=None, device=device)  # 注意：我们先只加载结构

# ==== 加载 checkpoint ====
state_dict = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()

# ==== 模拟一次前向输入以 trace 模型 ====
dummy_x0 = torch.rand(1, 3, 512, 512).to(device)
dummy_x1 = torch.rand(1, 3, 512, 512).to(device)
dummy_dt = torch.tensor([[0.5]], dtype=torch.float32).to(device)

with torch.no_grad():
    traced = torch.jit.trace(model, (dummy_x0, dummy_x1, dummy_dt))

# ==== 保存 TorchScript 模型 ====
traced.save(export_path)
print(f"✅ Exported to TorchScript: {export_path}")


d:\Projects\CV-Project\code\interpolation\frame-interpolation-pytorch\film_sft\util.py:78: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  normalized_flow2 = flow.permute(0, 2, 3, 1) / torch.tensor(
d:\Projects\CV-Project\code\interpolation\frame-interpolation-pytorch\film_sft\util.py:78: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  normalized_flow2 = flow.permute(0, 2, 3, 1) / torch.tensor(


✅ Exported to TorchScript: film_sft_last.pt
